In [2]:
import requests
from bs4 import BeautifulSoup
import json
from jsonpath_ng.ext import parse

In [3]:
def getJSONSection(json_data, section):
    return [obj for obj in json_data if obj['name'] == section]

In [4]:
def getJSONValue(json_data, section, stat):
    section_data = getJSONSection(json_data['splits']['categories'], section)
    stat_data = getJSONSection(section_data[0]['stats'], stat)
    return stat_data[0]['displayValue']

In [5]:
# todo - add the rest
def getTeamUrl(team_name):
    if (team_name == 'Real Sociedad'):
        return 'real-sociedad'
    if (team_name == 'Villarreal'):
        return 'villarreal-cf'
    if (team_name == 'Valencia'):
        return 'valencia-cf'
    if (team_name == 'Almería'):
        return 'ud-almeria'
    

    if (team_name == 'TSG Hoffenheim'):
        return 'tsg-hoffenheim'
    if (team_name == '1. FC Union Berlin'):
        return '1-fc-union-berlin'
    if (team_name == 'Borussia Dortmund'):
        return 'borussia-dortmund'
    if (team_name == 'Bayern Munich'):
        return 'fc-bayern-muenchen'
    

    if (team_name == 'Pumas UNAM'):
        return "18/pumas"
    if (team_name == 'Monterrey'):
         return "14/monterrey"
    if (team_name == 'FC Juarez'):
         return "11790/fc-juarez"
    if (team_name == 'Pachuca'):
         return "11/pachuca"
    if (team_name == 'Querétaro'):
         return "12037/queretaro"
    if (team_name == 'Mazatlán FC'):
         return "12043/mazatlan-fc"


TODO:

Add pagination
Don't store goalkeeper stats for non-goalkeepers

In [7]:
# premier league
athlete_list_results = requests.get('http://sports.core.api.espn.com/v2/sports/soccer/leagues/eng.1/seasons/2023/athletes?lang=en&region=us&limit=1000')

# la liga
#athlete_list_results = requests.get('http://sports.core.api.espn.com/v2/sports/soccer/leagues/esp.1/seasons/2023/athletes?lang=en&region=us&limit=1000')

# bundesliga
#athlete_list_results = requests.get('http://sports.core.api.espn.com/v2/sports/soccer/leagues/ger.1/seasons/2023/athletes?lang=en&region=us&limit=1000')

# mls
#athlete_list_results = requests.get('http://sports.core.api.espn.com/v2/sports/soccer/leagues/usa.1/seasons/2023/athletes?lang=en&region=us&limit=1000')

# nwsl
#athlete_list_results = requests.get('http://sports.core.api.espn.com/v2/sports/soccer/leagues/usa.nwsl/seasons/2023/athletes?lang=en&region=us&limit=1000')

# liga mx
#athlete_list_results = requests.get('http://sports.core.api.espn.com/v2/sports/soccer/leagues/mex.1/seasons/2023/athletes?lang=en&region=us&limit=1000')

# france
#athlete_list_results = requests.get('http://sports.core.api.espn.com/v2/sports/soccer/leagues/fra.1/seasons/2023/athletes?lang=en&region=us&limit=1000')

# italy


athlete_list_results_json = athlete_list_results.json()

for athlete in range(10):
    athlete_results = requests.get(athlete_list_results_json['items'][athlete]['$ref'])
    athlete_results_json = athlete_results.json()
    if ('statistics' in athlete_results_json):
        print('**********')
        print(f"Name: {athlete_results_json['displayName']}")
        print(f"Weight: {athlete_results_json['displayWeight'] if 'displayWeight' in athlete_results_json else 'N/A'}")
        print(f"Height: {athlete_results_json['displayHeight'] if 'displayHeight' in athlete_results_json else 'N/A'}")
        print(f"Nationality: {athlete_results_json['citizenship']}")
        print(f"Position: {athlete_results_json['position']['displayName']}")
        print(f"Birthdate: {athlete_results_json['dateOfBirth']}")
        print(f"Jersey Number: {athlete_results_json['jersey']}")

        team_results = requests.get(athlete_results_json['team']['$ref'])
        team_results_json = team_results.json()
        print (f"Team: {team_results_json['displayName']}")

        athlete_stats_results = requests.get(athlete_results_json['statistics']['$ref'])
        athlete_stats_results_json = athlete_stats_results.json()
        
        print(f"Goals: {getJSONValue(athlete_stats_results_json, 'offensive', 'totalGoals')}")
        print(f"Assists: {getJSONValue(athlete_stats_results_json, 'offensive', 'goalAssists')}")
        print(f"Shots: {getJSONValue(athlete_stats_results_json, 'offensive', 'totalShots')}")
        print(f"Shots on Goal: {getJSONValue(athlete_stats_results_json, 'offensive', 'shotsOnTarget')}")

        print(f"Yellow Cards: {getJSONValue(athlete_stats_results_json, 'general', 'yellowCards')}")
        print(f"Red Cards: {getJSONValue(athlete_stats_results_json, 'general', 'redCards')}")

        print(f"Saves: {getJSONValue(athlete_stats_results_json, 'goalKeeping', 'saves')}")
        print(f"Clean Sheets: {getJSONValue(athlete_stats_results_json, 'goalKeeping', 'cleanSheet')}")
        print(f"Goals Against: {getJSONValue(athlete_stats_results_json, 'goalKeeping', 'goalsConceded')}")

        #premier league
        name_search_url = f"https://footballapi.pulselive.com/search/PremierLeague/?terms={athlete_results_json['displayName']}&type=player&size=1&start=0&fullObjectResponse=true"
        name_search_result = requests.get(name_search_url, headers = {"origin": "https://www.premierleague.com"})
        name_search_result_json = name_search_result.json()

        image_id = name_search_result_json['hits']['hit'][0]['response']['altIds']['opta']
        image_url = f"https://resources.premierleague.com/premierleague/photos/players/250x250/{image_id}.png"

        print(image_url)

        #laliga
        # club_squad_url = f"https://www.laliga.com/en-US/clubs/{getTeamUrl(team_results_json['displayName'])}/squad"
        # club_squad_results = requests.get(club_squad_url)
        # club_squad_results_parsed = BeautifulSoup(club_squad_results.content.decode('utf-8'))
        # club_squad_json = json.loads(club_squad_results_parsed.body.find('script', attrs={'id': '__NEXT_DATA__'}).text)
        # player_element = [obj for obj in club_squad_json['props']['pageProps']['squad']['squads'] if ('shirt_number' in obj) and (str(obj['shirt_number'])) == athlete_results_json['jersey']]
        # image_url =  player_element[0]['photos']['002']['512x512']

        # print(image_url)
        
        # bundesliga
        # club_squad_url = f"https://www.bundesliga.com/en/bundesliga/clubs/{getTeamUrl(team_results_json['displayName'])}"
        # club_squad_results = requests.get(club_squad_url)
        # club_squad_results_parsed = BeautifulSoup(club_squad_results.content.decode('utf-8'))
        # club_squad_json = json.loads(club_squad_results_parsed.body.find('script', attrs={'id': 'serverApp-state'}).text)
        # jsonpath_expr = parse(f"$..players.*[?(@.shirtNumber={athlete_results_json['jersey']})]")
        # match = [items.value['playerImages']['FACE_CIRCLE'].replace('-circle', '') for items in jsonpath_expr.find(club_squad_json)]
        # image_url = match[0]

        # print(image_url)

        # mls
        #image_url = athlete_results_json['headshot']['href'] if 'headshot' in athlete_results_json else 'N/A'
        #print(image_url)

        # nwsl
        # base_url = 'https://d2nkt8hgeld8zj.cloudfront.net'
        # player_list_url = f'{base_url}/services/nwsl.ashx/players'

        # player_list_results = requests.get(player_list_url)
        # player_list_results_results_json = player_list_results.json()
        
        # jsonpath_expr = parse(f"$.data[?(@.shirtNumber=='{athlete_results_json['jersey']}' & @.team.title == '{team_results_json['displayName']}')]")
        # match = [items.value['images']['head_shot']['url'] for items in jsonpath_expr.find(player_list_results_results_json)]
        # image_url = f'{base_url}{match[0]}' if len(match) > 0 else 'N/A'
        # print(image_url)

        # liga mx
        # club_squad_url = f"https://ligamx.net/cancha/club/{getTeamUrl(team_results_json['displayName'])}"
        # club_squad_results = requests.get(club_squad_url)
        # club_squad_results_parsed = BeautifulSoup(club_squad_results.content.decode('utf-8'))

        # match = [number.parent.parent.parent.find('img') for number in club_squad_results_parsed.find_all('strong', {'class': 'numero'}) if number.get_text() == f"#{athlete_results_json['jersey']}"]
        # image_url = match[0]['src'] if len(match) > 0 else 'N/A'
        # print(image_url)

**********
Name: Michael Keane
Weight: 181 lbs
Height: 6' 3"
Nationality: England
Position: Defender
Birthdate: 1993-01-11T08:00Z
Jersey Number: 5
Team: Everton
Goals: 0
Assists: 0
Shots: 2
Shots on Goal: 2
Yellow Cards: 1
Red Cards: 0
Saves: 2
Clean Sheets: 0
Goals Against: 7
https://resources.premierleague.com/premierleague/photos/players/250x250/p106611.png
**********
Name: Andy Lonergan
Weight: 192 lbs
Height: 6' 4"
Nationality: England
Position: Goalkeeper
Birthdate: 1983-10-19T07:00Z
Jersey Number: 31
Team: Everton
Goals: 0
Assists: 0
Shots: 0
Shots on Goal: 0
Yellow Cards: 0
Red Cards: 0
Saves: 0
Clean Sheets: 0
Goals Against: 0
https://resources.premierleague.com/premierleague/photos/players/250x250/p11948.png
**********
Name: Ben Davies
Weight: 170 lbs
Height: 5' 11"
Nationality: Wales
Position: Defender
Birthdate: 1993-04-24T07:00Z
Jersey Number: 33
Team: Tottenham Hotspur
Goals: 0
Assists: 0
Shots: 2
Shots on Goal: 0
Yellow Cards: 0
Red Cards: 0
Saves: 2
Clean Sheets: 0
Goal